# 경로 설정

In [32]:
path = '../data/'
import os
os.listdir(path)

['train.csv', 'test.csv']

# Data Load

In [33]:
import pandas as pd

train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')

In [34]:
print(train.shape)
print(test.shape)

(15000, 70)
(35452, 69)


In [35]:
train.head()

,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
0,0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,...,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,...,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,...,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,...,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,...,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0


In [36]:
train.drop(columns=['index','country'], inplace = True)
test.drop(columns=['index','country'], inplace = True)

# 실험 환경 구축

In [6]:
import jinja2
from pycaret.classification import *

/home/twogudack/anaconda3/envs/pytorch/lib/python3.6/site-packages/pycaret/loggers/mlflow_logger.py:14: FutureWarning: MLflow support for Python 3.6 is deprecated and will be dropped in an upcoming release. At that point, existing Python 3.6 workflows that use MLflow will continue to work without modification, but Python 3.6 users will no longer get access to the latest MLflow features and bugfixes. We recommend that you upgrade to Python 3.7 or newer.
  import mlflow


In [37]:
clf = setup(data = train, target = 'nerdiness')

,Description,Value
0,session_id,3215
1,Target,nerdiness
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(15000, 68)"
5,Missing Values,True
6,Numeric Features,4
7,Categorical Features,63
8,Ordinal Features,False
9,High Cardinality Features,False


# 모델 3개 제작

In [89]:
best_3 = compare_models(sort = 'AUC', n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.7651,0.8559,0.8277,0.7658,0.7954,0.5207,0.5230,0.4050
rf,Random Forest Classifier,0.7637,0.8506,0.8317,0.7619,0.7952,0.5172,0.5202,0.3040
lightgbm,Light Gradient Boosting Machine,0.7425,0.8170,0.8049,0.7478,0.7752,0.4749,0.4768,0.0660
gbc,Gradient Boosting Classifier,0.7314,0.7995,0.7978,0.7370,0.7661,0.4518,0.4540,0.6330
lda,Linear Discriminant Analysis,0.7219,0.7936,0.7800,0.7330,0.7556,0.4336,0.4352,0.1740
ada,Ada Boost Classifier,0.7196,0.7902,0.7749,0.7325,0.7529,0.4293,0.4305,0.1610
lr,Logistic Regression,0.7196,0.7891,0.7793,0.7305,0.7540,0.4287,0.4302,0.7790
dt,Decision Tree Classifier,0.6872,0.6835,0.7191,0.7156,0.7172,0.3673,0.3675,0.0610
nb,Naive Bayes,0.4487,0.6221,0.0045,0.5310,0.0089,-0.0002,-0.0017,0.0270
knn,K Neighbors Classifier,0.5861,0.6032,0.6682,0.6147,0.6402,0.1548,0.1557,0.2240


# 모델 앙상블

In [91]:
blended = blend_models(estimator_list = best_3, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7757,0.8561,0.8351,0.7755,0.8042,0.5426,0.5446
1,0.7524,0.8427,0.8109,0.7573,0.7832,0.4954,0.4969
2,0.7667,0.8586,0.8421,0.7607,0.7993,0.5223,0.5262
3,0.7667,0.8561,0.8171,0.7731,0.7945,0.5251,0.5262
4,0.7713,0.8445,0.8351,0.7699,0.8012,0.5331,0.5356
Mean,0.7665,0.8516,0.8280,0.7673,0.7965,0.5237,0.5259
Std,0.0078,0.0066,0.0119,0.0071,0.0074,0.0158,0.0160


In [93]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7883,0.8723,0.8447,0.7902,0.8166,0.5669,0.5686


#  전체 데이터에 대한 재학습

In [94]:
final_model = finalize_model(blended)

# test set에 대한 예측

In [95]:
predictions = predict_model(final_model, data = test)

In [96]:
predictions

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,age,hand,religion,orientation,voted,married,familysize,ASD,Label,Score
0,4.0,4.0,3.0,5.0,5.0,5.0,3.0,5.0,4.0,5.0,...,19,1.0,4.0,4.0,1.0,1.0,3.0,2.0,0,0.6821
1,4.0,5.0,4.0,4.0,5.0,4.0,5.0,5.0,5.0,4.0,...,33,1.0,1.0,5.0,2.0,1.0,5.0,2.0,1,0.8003
2,5.0,5.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0,4.0,...,13,1.0,4.0,5.0,2.0,1.0,3.0,2.0,1,0.8258
3,5.0,4.0,3.0,4.0,5.0,4.0,5.0,4.0,4.0,5.0,...,28,1.0,2.0,2.0,2.0,1.0,3.0,2.0,1,0.5770
4,5.0,5.0,5.0,5.0,5.0,3.0,5.0,5.0,5.0,5.0,...,15,1.0,4.0,5.0,2.0,1.0,2.0,2.0,1,0.8182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35447,4.0,5.0,5.0,3.0,3.0,4.0,5.0,5.0,4.0,3.0,...,16,3.0,10.0,4.0,2.0,1.0,3.0,2.0,1,0.8195
35448,5.0,5.0,5.0,5.0,5.0,4.0,5.0,3.0,5.0,3.0,...,16,3.0,2.0,5.0,2.0,1.0,2.0,2.0,1,0.7701
35449,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,31,1.0,1.0,5.0,1.0,1.0,2.0,2.0,1,0.9342
35450,5.0,5.0,4.0,5.0,5.0,1.0,5.0,1.0,5.0,5.0,...,19,1.0,12.0,1.0,2.0,1.0,NaN,2.0,0,0.7234


In [86]:
submission = pd.read_csv(path + 'submission.csv')

In [87]:
submission['nerdiness'] = predictions['Label']

In [88]:
submission.to_csv('submission_proba.csv', index = False)